In [46]:
import pandas as pd
from tqdm import tqdm
from collections import Counter
from tqdm import tqdm
import random
from scipy import sparse
import numpy as np
import pandas as pd
from sklearn import preprocessing
import time
import tensorflow as tf

In [2]:
train_pd = pd.read_csv("./data/train.csv", encoding="utf-8")

In [3]:
word_list = []
vocab_list = []
for line in tqdm(train_pd.value):
    word_list.append([w for w in line])
    vocab_list.extend([w for w in line])

100%|█████████████████████████████████████████████████████████████████████████| 30000/30000 [00:00<00:00, 93492.77it/s]


In [4]:
c = Counter(vocab_list)
vocab = np.array(list(c.keys()))
vocab.sort()
vocab

array(['%', '&', '*', '+', '-', '.', '0', '1', '2', '3', '4', '5', '6',
       '7', '8', '9', '=', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I',
       'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V',
       'W', 'X', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h',
       'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
       'v', 'w', 'x', 'y', 'z'],
      dtype='<U1')

In [5]:
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(vocab)
label_encoder.classes_
label_encoder.transform(vocab)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69], dtype=int64)

In [6]:
word_labelencoder = [list(label_encoder.transform(w)) for w in word_list]
word_size = [len(i) for i in word_labelencoder]

In [7]:
train_size_pd = pd.DataFrame(word_size, columns=['len'])
train_size_pd['label'] = train_pd.label
train_size_pd.describe()

,len,label
count,30000.000000,30000.000000
mean,113.928533,0.083333
std,173.427420,0.276390
min,16.000000,0.000000
25%,53.000000,0.000000
50%,73.000000,0.000000
75%,100.000000,0.000000
max,11553.000000,1.000000


In [8]:
train_size_pd = train_size_pd[train_size_pd.len < 500]
Counter(train_size_pd.label)

Counter({0: 27159, 1: 1973})

In [9]:
pos_index = train_size_pd[train_size_pd.label == 1].index.values
random.shuffle(pos_index)
pos_index, pos_index.shape

(array([ 7265, 19628, 14267, ..., 16807,  5813, 22676], dtype=int64), (1973,))

In [10]:
neg_index = train_size_pd[train_size_pd.label == 0].index.values
neg_index = np.random.choice(neg_index, size=4000-1973)
neg_index, neg_index.shape

(array([26665,  3689, 12644, ...,  5089, 29843,  6239], dtype=int64), (2027,))

In [11]:
balance_train_index = np.append(pos_index,neg_index)
random.shuffle(balance_train_index)
balance_train_index, balance_train_index.shape

(array([23528,  5735,  9397, ...,  9912,  2248, 24500], dtype=int64), (4000,))

In [12]:
x_batch = np.array(word_labelencoder.copy())
x_batch = x_batch[balance_train_index]
y_batch = np.array(train_pd.label.copy())
y_batch = y_batch[balance_train_index]
x_batch_size = [len(i) for i in x_batch]
x_batch.shape, y_batch.shape, len(x_batch_size)

((4000,), (4000,), 4000)

In [13]:
x_batch[:1]

array([ list([54, 48, 68, 58, 57, 48, 16, 4, 7, 13, 11, 7, 0, 8, 11, 8, 11, 0, 8, 11, 8, 13, 0, 8, 11, 8, 6, 37, 30, 25, 31, 30, 0, 8, 11, 8, 6, 17, 28, 28, 0, 8, 11, 8, 6, 35, 21, 28, 21, 19, 36, 0, 8, 11, 8, 6, 9, 12, 10, 13, 0, 8, 11, 8, 19, 9, 12, 10, 13, 0, 8, 11, 8, 19, 9, 12, 10, 13, 0, 8, 11, 8, 19, 9, 12, 10, 13, 0, 8, 11, 8, 19, 9, 12, 10, 13, 0, 8, 11, 8, 19, 9, 12, 10, 13, 0, 8, 11, 8, 19, 9, 12, 10, 13, 0, 8, 11, 8, 19, 9, 12, 10, 13, 0, 8, 11, 8, 19, 9, 12, 10, 13, 0, 8, 11, 8, 19, 9, 12, 10, 13, 0, 8, 11, 8, 19, 9, 12, 10, 13, 0, 8, 11, 8, 19, 9, 12, 10, 13, 0, 8, 11, 8, 19, 9, 12, 10, 13, 0, 8, 11, 8, 19, 9, 12, 10, 13, 0, 8, 11, 8, 19, 9, 12, 10, 13, 0, 8, 11, 8, 19, 9, 12, 10, 13, 0, 8, 11, 8, 19, 9, 12, 10, 13, 0, 8, 11, 8, 19, 9, 12, 10, 13, 0, 8, 11, 8, 19, 9, 12, 10, 13, 0, 8, 11, 8, 19, 9, 12, 10, 13, 0, 8, 11, 8, 19, 9, 12, 10, 13, 0, 8, 11, 8, 19, 9, 12, 10, 13, 0, 8, 11, 8, 19, 9, 12, 10, 13, 0, 8, 11, 8, 19, 9, 12, 10, 13, 0, 8, 11, 8, 19, 9, 12, 10, 13, 0, 8

In [14]:
y_batch[:100]

array([1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 0, 0, 0], dtype=int64)

In [15]:
max_size = np.max(x_batch_size)
max_size

499

In [31]:
x_batch_pad = []
for x in tqdm(x_batch[:]):
    list_test = list()
    list_test =([-1] * max_size)
    list_test[:len(x)] = x
#     list_test = [[i] for i in list_test]
    x_batch_pad.append(list_test)

100%|██████████████████████████████████████████████████████████████████████████| 4000/4000 [00:00<00:00, 117337.96it/s]


In [32]:
x_batch_pad[:10]

[[54,
  48,
  68,
  58,
  57,
  48,
  16,
  4,
  7,
  13,
  11,
  7,
  0,
  8,
  11,
  8,
  11,
  0,
  8,
  11,
  8,
  13,
  0,
  8,
  11,
  8,
  6,
  37,
  30,
  25,
  31,
  30,
  0,
  8,
  11,
  8,
  6,
  17,
  28,
  28,
  0,
  8,
  11,
  8,
  6,
  35,
  21,
  28,
  21,
  19,
  36,
  0,
  8,
  11,
  8,
  6,
  9,
  12,
  10,
  13,
  0,
  8,
  11,
  8,
  19,
  9,
  12,
  10,
  13,
  0,
  8,
  11,
  8,
  19,
  9,
  12,
  10,
  13,
  0,
  8,
  11,
  8,
  19,
  9,
  12,
  10,
  13,
  0,
  8,
  11,
  8,
  19,
  9,
  12,
  10,
  13,
  0,
  8,
  11,
  8,
  19,
  9,
  12,
  10,
  13,
  0,
  8,
  11,
  8,
  19,
  9,
  12,
  10,
  13,
  0,
  8,
  11,
  8,
  19,
  9,
  12,
  10,
  13,
  0,
  8,
  11,
  8,
  19,
  9,
  12,
  10,
  13,
  0,
  8,
  11,
  8,
  19,
  9,
  12,
  10,
  13,
  0,
  8,
  11,
  8,
  19,
  9,
  12,
  10,
  13,
  0,
  8,
  11,
  8,
  19,
  9,
  12,
  10,
  13,
  0,
  8,
  11,
  8,
  19,
  9,
  12,
  10,
  13,
  0,
  8,
  11,
  8,
  19,
  9,
  12,
  10,
  13,
  0,
  8,
  11,


In [33]:
x_batch_pad = [np.eye(len(vocab))[item] for item in x_batch_pad]
# x_batch_pad = [list(i) for i in x_batch_pad]

In [34]:
x_batch_pad[0].shape

(499, 70)

In [35]:
x_batch_pad[:1]

[array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  1.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.]])]

In [36]:
y_batch_pad = [np.eye(2)[item] for item in y_batch]
y_batch_pad = [list(i) for i in y_batch_pad]

In [37]:
y_batch_pad

[[0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [1.0, 0.0],

In [38]:
x_batch_size

[497,
 57,
 394,
 178,
 34,
 41,
 206,
 86,
 170,
 149,
 385,
 224,
 177,
 71,
 405,
 238,
 70,
 373,
 82,
 110,
 77,
 100,
 83,
 175,
 63,
 261,
 40,
 25,
 499,
 87,
 77,
 49,
 241,
 158,
 105,
 76,
 180,
 67,
 87,
 85,
 417,
 72,
 305,
 319,
 136,
 260,
 120,
 110,
 81,
 132,
 158,
 34,
 114,
 172,
 308,
 477,
 149,
 426,
 66,
 221,
 56,
 160,
 87,
 126,
 28,
 213,
 130,
 91,
 289,
 102,
 208,
 230,
 430,
 80,
 73,
 55,
 69,
 85,
 72,
 209,
 56,
 80,
 108,
 296,
 57,
 50,
 121,
 323,
 272,
 348,
 491,
 104,
 383,
 288,
 70,
 199,
 67,
 117,
 101,
 64,
 41,
 77,
 269,
 175,
 48,
 417,
 55,
 193,
 37,
 84,
 35,
 135,
 35,
 61,
 352,
 328,
 232,
 64,
 263,
 55,
 68,
 253,
 112,
 314,
 228,
 104,
 76,
 191,
 34,
 122,
 86,
 382,
 367,
 45,
 67,
 255,
 87,
 301,
 76,
 80,
 372,
 90,
 91,
 96,
 370,
 440,
 42,
 144,
 60,
 355,
 41,
 52,
 76,
 211,
 244,
 176,
 449,
 120,
 105,
 119,
 40,
 308,
 114,
 93,
 269,
 83,
 133,
 59,
 459,
 236,
 77,
 437,
 85,
 49,
 245,
 120,
 42,
 205,
 130,
 6

In [39]:
class Batcher():
    def __init__(self, x, y, x_batch_size):
        self.split_size = int(len(x)*0.8)
        self.train_x = x[:self.split_size]
        self.train_y = y[:self.split_size]
        self.train_size = x_batch_size[:self.split_size]
        self.test_x = x[self.split_size:]
        self.test_y = y[self.split_size:]
        self.test_size = x_batch_size[self.split_size:]
        self.start = 0
    def next_batch(self, batch_size):
        s_index = self.start
        e_index = self.start + batch_size
        if e_index >= self.split_size:
            self.start = 0
            s_index = self.start
            e_index = self.start + batch_size
        self.start = e_index
        return self.train_x[s_index:e_index], self.train_y[s_index:e_index], self.train_size[s_index:e_index]

In [40]:
sqli_batch = Batcher(x_batch_pad, y_batch_pad, x_batch_size)

In [41]:
batch_data, batch_labels, batch_seqlen = sqli_batch.next_batch(10)

In [42]:
batch_data

[array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  1.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.]]),
 array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  1.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.]]),
 array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  1.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.]]),
 array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       

In [43]:
batch_labels

[[0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0]]

In [44]:
batch_seqlen

[497, 57, 394, 178, 34, 41, 206, 86, 170, 149]

In [50]:
tf.reset_default_graph()

# Parameters
learning_rate = 0.01
training_steps = 50
batch_size = 100
display_step = 100

# Network Parameters
seq_max_len = max_size # Sequence max length
n_hidden = 128 # hidden layer num of features
n_classes = 2 # linear sequence or not

# trainset = ToySequenceData(n_samples=1000, max_seq_len=seq_max_len)
# testset = ToySequenceData(n_samples=500, max_seq_len=seq_max_len)
sqli_batch = Batcher(x_batch_pad, y_batch_pad, x_batch_size)

# tf Graph input
x = tf.placeholder("float", [None, seq_max_len, len(vocab)])
y = tf.placeholder("float", [None, n_classes])
# A placeholder for indicating each sequence length
seqlen = tf.placeholder(tf.int32, [None])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}


def dynamicRNN(x, seqlen, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, seq_max_len, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)

    # Get lstm cell output, providing 'sequence_length' will perform dynamic
    # calculation.
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32,
                                sequence_length=seqlen)

    # When performing dynamic calculation, we must retrieve the last
    # dynamically computed output, i.e., if a sequence length is 10, we need
    # to retrieve the 10th output.
    # However TensorFlow doesn't support advanced indexing yet, so we build
    # a custom op that for each sample in batch size, get its length and
    # get the corresponding relevant output.

    # 'outputs' is a list of output at every timestep, we pack them in a Tensor
    # and change back dimension to [batch_size, n_step, n_input]
    outputs = tf.stack(outputs)
    outputs = tf.transpose(outputs, [1, 0, 2])

    # Hack to build the indexing and retrieve the right output.
    batch_size = tf.shape(outputs)[0]
    # Start indices for each sample
    index = tf.range(0, batch_size) * seq_max_len + (seqlen - 1)
    # Indexing
    outputs = tf.gather(tf.reshape(outputs, [-1, n_hidden]), index)

    # Linear activation, using outputs computed above
    return tf.matmul(outputs, weights['out']) + biases['out']

pred = dynamicRNN(x, seqlen, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps + 1):
#         batch_x, batch_y, batch_seqlen = trainset.next(batch_size)
        batch_x, batch_y, batch_seqlen = sqli_batch.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       seqlen: batch_seqlen})
        if step % display_step == 0 or step == 1:
            # Calculate batch accuracy & loss
            acc, loss = sess.run([accuracy, cost], feed_dict={x: batch_x, y: batch_y,
                                                seqlen: batch_seqlen})
            print("Step " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy
#     test_data = testset.data
#     test_label = testset.labels
#     test_seqlen = testset.seqlen
    test_data = sqli_batch.test_x
    test_label = sqli_batch.test_y
    test_seqlen = sqli_batch.test_size
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label,
                                      seqlen: test_seqlen}))

c:\program files\python36\lib\site-packages\tensorflow\python\ops\gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Step 100, Minibatch Loss= 0.725667, Training Accuracy= 0.49000
Step 10000, Minibatch Loss= 0.473397, Training Accuracy= 0.78000
Step 20000, Minibatch Loss= 0.276774, Training Accuracy= 0.94000
Step 30000, Minibatch Loss= 0.073246, Training Accuracy= 0.98000
Step 40000, Minibatch Loss= 0.115629, Training Accuracy= 0.97000
Step 50000, Minibatch Loss= 0.058722, Training Accuracy= 0.99000
Step 60000, Minibatch Loss= 0.095137, Training Accuracy= 0.98000
Step 70000, Minibatch Loss= 0.107541, Training Accuracy= 0.97000
Step 80000, Minibatch Loss= 0.039399, Training Accuracy= 0.99000
Step 90000, Minibatch Loss= 0.131407, Training Accuracy= 0.96000
Step 100000, Minibatch Loss= 0.081308, Training Accuracy= 0.98000
Step 110000, Minibatch Loss= 0.087308, Training Accuracy= 0.98000
Step 120000, Minibatch Loss= 0.090227, Training Accuracy= 0.98000
Step 130000, Minibatch Loss= 0.125701, Training Accuracy= 0.97000
Step 140000, Minibatch Loss= 0.165360, Training Accuracy= 0.94000
Step 150000, Minibatch